# Read, clean, merge, and write PLIC facets

The Basilisk simulations write PLIC facecets to the folder *plic/* of each simulation. The PLIC points are written after **10** iterations. Each process stores the points in a separate file. The files are named as

points_**iteration**_n**process**.txt

For example, the file of process 8 after 120 iterations is called

points_**000120**_n**008**.txt

Each file contains the all points in Cartesian coordinates. The points belonging to one PLIC facet are separated by one empty line. In the following example, the first facet consists of **three** and the second of **four** vertices:
```
# the order is px / py / pz
-0.472504 3.39844 -0.0585938 # first point of facet one
-0.46875 3.38866 -0.0585938
-0.46875 3.39844 -0.0823362

-0.474992 3.39844 -0.0292969 # first point of facet two
-0.46875 3.38206 -0.0292969
-0.46875 3.38931 -0.0585938
-0.47223 3.39844 -0.0585938
...
```
In two-dimensional data sets the entry **pz** is missing. For 2D cases, pass *shape_2D=True* as argument to the *process_shape* function

In [1]:
# paths to define from where to read and where to write the processed data
source_base = "../data"
target_base = "../data"
# the processed data will be stored in the newly created folder plic_clean
plic_source = "plic"
plic_target = "plic_clean"

#cases = (["dell_l{:02d}".format(level) for level in [15, 16, 17]] +
#         ["wa18_l{:02d}".format(level) for level in [15, 16]] +
#         ["scap_l{:02d}".format(level) for level in [15, 16]])
cases = (["wa18_l{:02d}".format(level) for level in [16]] +
         ["scap_l{:02d}".format(level) for level in [15, 16]])

for i, case in enumerate(cases):
    print(i, case)

0 wa18_l16
1 scap_l15
2 scap_l16


In [2]:
import glob
import pandas as pd
import numpy as np
    

def assemble_shape(path, iteration, shape_2D=False):
    '''Read PLIC intersections points from disk.
       The function reads all avaialable processor files und concatenates them.
        
    Parameters
    ----------
    path - string: path to the file location
    iteration - integer: iteration to load
    
    Returns
    -------
    facets - DataFrame: DataFrame containing the x, y, z coordintes and
             the number of the facet to which a pints belongs
             
    '''
    base_name = path + "/points_{:06d}_n".format(iteration)
    files = sorted(glob.glob(base_name + "*"))
    points = []
    if shape_2D:
        columns = ["px", "py"]
    else:
        columns = ["px", "py", "pz"]
    for file in files:
        points.append(pd.read_csv(file, sep=" ", names=columns, engine='c', dtype=np.float32))
    all_points = pd.concat(points)
    all_points.dropna(inplace=True)
    return all_points.reset_index(drop=True)


def get_iterations(path):
    ''' Find all iterations based on the file names.
    
    Parameters
    ----------
    path - string : where to search for files
    
    Returns
    -------
    iterations - array-like : set of all iterations
    
    '''
    file_paths = glob.glob(path + "/*_n000.txt")
    iterations = sorted([int(path.split("/")[-1].split("_")[1]) for path in file_paths])
    return iterations


def process_shape(source, target, iteration, shape_2D):
    '''Read, assemble, and write PLIC data.
    '''
    data = assemble_shape(source, iteration, shape_2D)
    file_name = "plic_{:06d}.pkl".format(iteration)
    data.to_pickle(target + "/" + file_name)

In [3]:
import os
from joblib import Parallel, delayed
from tqdm import tqdm_notebook


for case in cases:
    print("Processing simulation folder {}".format(case))
    source = source_base + "/" + case + "/" + plic_source
    iterations = get_iterations(source)
    target = target_base + "/" + case + "/" + plic_target
    if not os.path.exists(target):
        os.makedirs(target)
    Parallel(n_jobs=4)(delayed(process_shape)(source, target, it, shape_2D=True) for it in tqdm_notebook(iterations))

Processing simulation folder wa18_l16



Processing simulation folder scap_l15



Processing simulation folder scap_l16
